In [1]:
import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
TARGET_LEAGUES = [
    'LTC', 'LCK',
    'LPL',
    'EU LCS', 'LEC',
    'NA LCS', 'LCS',
    'GPL', 'LST',
    'LLA', 'CLS', 'LLN',
    'LCO', 'OPL',
    'LCL',
    'LJL',
    'LMS', 'PCS',
    'VCS',
    'CBLOL', 
    'TCL',
    
    'MSI',
    'WCS',
]

In [3]:
def get_new_id(ids):
    if 'team_id' in ids.columns:
        id_list = sorted(ids['team_id'].unique())
    else:
        id_list = sorted(ids['player_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

## Teams

In [4]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [5]:
lst = []
for league in TARGET_LEAGUES:
    if league not in leagues['League Short'].values:
        lst.append(league)
lst

[]

In [22]:
teams = pd.read_csv('./csv/teams_id.csv')
names = []

# for league in TARGET_LEAGUES:
for league in leagues['League Short']:
    print(league)
    tournaments = get_tournaments(where=f'L.League_Short="{league}"')
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        try:
            scoreboard_games = get_scoreboard_games(
                where=f'T.OverviewPage="{page}"'
            )
            scoreboard_players = get_scoreboard_players(
                where=f'T.OverviewPage="{page}"'
            )
        except Exception as e:
            print(e)
            continue
        if scoreboard_games is not None:
            scoreboard_games = scoreboard_games.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
        if scoreboard_players is not None:
            scoreboard_players = scoreboard_players.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_players['Team'].unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
    if len(names) > 0:
        print(scoreboard_games['OverviewPage'].unique()[0])
        print(scoreboard_players['OverviewPage'].unique()[0])
        print(names)
        break

2015 IWCT
	None
'OverviewPage'
IWCQ
	2016 International Wildcard Qualifier
CBLOL
	CBLOL/2014 Season/Champions Series
	CBLOL/2014 Season/Champions Series/Playoffs
	CBLOL/2015 Season/Split 1 Qualifiers
	CBLOL/2015 Season/Split 1
	CBLOL/2015 Season/Split 1 Playoffs
	CBLOL/2015 Season/Split 2 Promotion
	CBLOL/2015 Season/Split 2
	CBLOL/2015 Season/Split 2 Playoffs
	CBLOL/2015 Season/Post-Season
	CBLOL/2016 Season/Split 1 Promotion
	CBLOL/2016 Season/Split 1
	CBLOL/2016 Season/Split 1 Playoffs
	CBLOL/2016 Season/Split 2 Promotion
	CBLOL/2016 Season/Split 2
	CBLOL/2016 Season/Split 2 Playoffs
	CBLOL/2017 Season/Split 1 Promotion
	CBLOL/2017 Season/Split 1
	CBLOL/2017 Season/Split 2 Promotion
	CBLOL/2017 Season/Split 1 Playoffs
	CBLOL/2017 Season/Split 2
	CBLOL/2017 Season/Split 2 Playoffs
	CBLOL/2018 Season/Split 1 Promotion
	CBLOL/2018 Season/Split 1
	CBLOL/2018 Season/Split 1 Playoffs
	CBLOL/2018 Season/Split 2 Promotion and Access
	CBLOL/2018 Season/Split 2
	CBLOL/2018 Season/Split 2 Play

In [18]:
teams.loc[teams['team'].str.contains('g2', case=False)]

,team,team_id
210,G2 Esports,162


In [19]:
if len(names) > 0:
    teams = pd.concat(
        [
            teams,
            pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
            # pd.Series({'team': names.pop(0), 'team_id': 259}).to_frame().T
        ],
        ignore_index=True
    )
names

[]

In [20]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

,team,team_id
0,100 Thieves,253
1,17 Academy,586
2,1UP Gaming,202
3,2144 Danmu Gaming,91
4,269 Gaming,506
...,...,...
709,n!faculty,168
710,paiN Gaming,262
711,team propertybb2,139
712,vVv Gaming,213


In [21]:
teams.to_csv('./csv/teams_id.csv', index=False)

## Players

In [1075]:
players = pd.read_csv('./csv/players_id.csv')
names = {}

for league in leagues['League Short']:
    print(league)
    tournaments = get_tournaments(where=f'L.League_Short="{league}"')
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        try:
            scoreboard_players = get_scoreboard_players(
                where=f'T.OverviewPage="{page}"'
            )
            if scoreboard_players is None:
                continue
        except Exception as e:
            print(e)
            continue
        scoreboard_players = scoreboard_players.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        player_names = list(set(scoreboard_players[['Link', 'Team']].itertuples(index=False)))
        names = {}
        for (name, team) in player_names:
            if name not in players['player'].values:
                lst = names.get(team, [])
                lst.append(name)
                names[team] = lst
        if len(names) > 0:
            break
    if len(names) > 0:
        print(scoreboard_players['OverviewPage'].unique()[0])
        for team, player_lst in names.items():
            print(f'{team}\n  {player_lst}')
        break

2015 IWCT
	None
IWCQ
	2016 International Wildcard Qualifier
CBLOL
	CBLOL/2014 Season/Champions Series
	CBLOL/2014 Season/Champions Series/Playoffs
	CBLOL/2015 Season/Split 1 Qualifiers
	CBLOL/2015 Season/Split 1
	CBLOL/2015 Season/Split 1 Playoffs
	CBLOL/2015 Season/Split 2 Promotion
	CBLOL/2015 Season/Split 2
	CBLOL/2015 Season/Split 2 Playoffs
	CBLOL/2015 Season/Post-Season
	CBLOL/2016 Season/Split 1 Promotion
	CBLOL/2016 Season/Split 1
	CBLOL/2016 Season/Split 1 Playoffs
	CBLOL/2016 Season/Split 2 Promotion
	CBLOL/2016 Season/Split 2
	CBLOL/2016 Season/Split 2 Playoffs
	CBLOL/2017 Season/Split 1 Promotion
	CBLOL/2017 Season/Split 1
	CBLOL/2017 Season/Split 2 Promotion
	CBLOL/2017 Season/Split 1 Playoffs
	CBLOL/2017 Season/Split 2
	CBLOL/2017 Season/Split 2 Playoffs
	CBLOL/2018 Season/Split 1 Promotion
	CBLOL/2018 Season/Split 1
	CBLOL/2018 Season/Split 1 Playoffs
	CBLOL/2018 Season/Split 2 Promotion and Access
	CBLOL/2018 Season/Split 2
	CBLOL/2018 Season/Split 2 Playoffs
	CBLOL/201

In [983]:
print(scoreboard_players['OverviewPage'].unique()[0])
teams = list(names.keys())
print(teams)
print()
print(teams[0])
print(names[teams[0]])

EU LCS/Season 3/Summer Promotion
['Sinners Never Sleep', 'Team ALTERNATE', 'Giants Gaming', 'Anexis eSports', 'MeetYourMakers', 'against All authority', 'Wizards e-Sports Club', 'Esport4Life', 'mousesports', 'TCM Gaming', 'DragonBorns']

Sinners Never Sleep
['Tabzz', 'Noonia', 'Nukeduck', 'wewillfailer', 'Zorozero']


In [1070]:
players.loc[players['player'].str.contains('yamato', case=False)]

,player,player_id
453,Yamatocannon,400


In [1072]:
if len(names) > 0:
    name = names[teams[0]].pop(0)
    players = pd.concat(
        [
            players,
            pd.Series({'player': name, 'player_id': get_new_id(players)}).to_frame().T
            # pd.Series({'player': name, 'player_id': 400}).to_frame().T
        ],
        ignore_index=True
    )
    if len(names[teams[0]]) == 0:
        del names[teams[0]]
        del teams[0]
print(teams)
print()
if len(teams) > 0:
    print(teams[0])
    print(names[teams[0]])

[]



In [1073]:
players = players.sort_values(by='player').reset_index(drop=True)
players

,player,player_id
0,1984,325
1,4LaN,132
2,7ico,77
3,Abagnale,436
4,Absolut,141
...,...,...
570,xPeke,409
571,xaiyen,438
572,yeTz,63
573,yellowpete,417


In [1074]:
players.to_csv('./csv/players_id.csv', index=False)

## Scoreboard

In [23]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [24]:
# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.DataFrame()
    # for league in TARGET_LEAGUES:
    for league in leagues['League Short']:
        t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
        tournaments = pd.concat([tournaments, t])
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape)

2023 (23, 17)


In [25]:
# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    print(year, tournaments.shape)
    scoreboard_games = pd.DataFrame()
    for page in tournaments['OverviewPage']:
        sg = get_scoreboard_games(where=f'T.OverviewPage="{page}"')
        if sg is None:
            print(f'{year}\t{page:35}\tdrop')
            tournaments.drop(tournaments.loc[tournaments['OverviewPage'] == page].index, inplace=True)
            continue
        league = tournaments.loc[tournaments['OverviewPage'] == page, 'League'].iloc[0]
        league = leagues.loc[leagues['League'] == league, 'League Short'].iloc[0]
        sg['League'] = league
        sg = sg.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        print(f'\t{sg.shape}\t{page}')
        scoreboard_games = pd.concat([scoreboard_games, sg])
    scoreboard_games.to_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv', index=False)
    print(f'{year} scoreboard_games {scoreboard_games.shape}')
    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(f'{year} tournaments {tournaments.shape}')

2023 (23, 17)
	(7, 37)	LLA/2023 Season/Opening Promotion
	(29, 37)	LPL/2023 Season/Spring Season
	(24, 37)	LCK/2023 Season/Spring Season
	(10, 37)	LEC/2023 Season/Winter Season
	(10, 37)	CBLOL/2023 Season/Split 1
2023	LCO/2023 Season/Split 1 Stage 1    	drop
2023	LLA/2023 Season/Opening Season     	drop
2023	TCL/2023 Season/Winter Season      	drop
2023	LCS/2023 Season/Spring Season      	drop
2023	LJL/2023 Season/Spring Season      	drop
2023	PCS/2023 Season/Spring Season      	drop
2023	LEC/2023 Season/Winter Groups      	drop
2023	LEC/2023 Season/Winter Playoffs    	drop
2023	LCO/2023 Season/Split 1 Stage 2    	drop
2023	LLA/2023 Season/Opening Playoffs   	drop
2023	LCK/2023 Season/Spring Playoffs    	drop
2023	2023 Mid-Season Invitational       	drop
2023	LEC/2023 Season/Season Finals      	drop
2023	LJL/2023 Season/Spring Playoffs    	drop
2023	PCS/2023 Season/Spring Playoffs    	drop
2023	LPL/2023 Season/Spring Playoffs    	drop
2023	VCS/2023 Season/Spring Season      	drop
2023	

In [26]:
scoreboard_games

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision,League
0,LLA/2023 Season/Opening Promotion,Incubus,XTEN Esports,XTEN Esports,Incubus,2022-09-10 20:09:00,0,1,2,29:46,29.766667,"Kalista,Gnar,LeBlanc,Sivir,Tahm Kench","Vi,Lucian,Renekton,Zeri,Aphelios","Gwen,Poppy,Taliyah,Jinx,Lulu","Fiora,Sejuani,Azir,Draven,Nautilus","JuanDeDios,Shall (Jorge Mendoza),Piqueos,Scena...","Zerito,Seize,Keine,Krim,Bardito",2.0,2.0,0.0,2.0,1.0,9.0,48256.0,60130.0,7.0,13.0,1.0,1.0,0.0,1.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_1_1,LLA/2023 Season/Opening Promotion_Qualifiers_1,None,0,LLA
1,LLA/2023 Season/Opening Promotion,XTEN Esports,Incubus,XTEN Esports,Incubus,2022-09-10 20:59:00,2,0,1,26:38,26.633333,"Vi,Lucian,Renekton,Jarvan IV,Wukong","Kalista,Fiora,Azir,Tahm Kench,Lulu","Ornn,Sejuani,Sylas,Jinx,Amumu","Gwen,Lee Sin,LeBlanc,Aphelios,Renata Glasc","Zerito,Seize,Keine,Krim,Bardito","JuanDeDios,Shall (Jorge Mendoza),Piqueos,Scena...",2.0,1.0,1.0,0.0,8.0,1.0,55042.0,44598.0,20.0,10.0,2.0,0.0,1.0,0.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_1_2,LLA/2023 Season/Opening Promotion_Qualifiers_1,None,0,LLA
2,LLA/2023 Season/Opening Promotion,Incubus,XTEN Esports,XTEN Esports,Incubus,2022-09-10 21:49:00,0,3,2,31:56,31.933333,"Kalista,Fiora,LeBlanc,Lulu,Irelia","Vi,Lucian,Renekton,Yuumi,Braum","Aatrox,Jarvan IV,Azir,Zeri,Bard","Camille,Sejuani,Sylas,Sivir,Renata Glasc","JuanDeDios,Shall (Jorge Mendoza),Piqueos,Scena...","Zerito,Seize,Keine,Krim,Bardito",1.0,3.0,0.0,2.0,0.0,11.0,50902.0,65286.0,5.0,14.0,0.0,2.0,0.0,4.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_1_3,LLA/2023 Season/Opening Promotion_Qualifiers_1,None,0,LLA
3,LLA/2023 Season/Opening Promotion,Globant Emerald,The Kings,The Kings,Globant Emerald,2022-09-11 20:08:00,0,1,2,30:36,30.600000,"Trundle,Azir,Yuumi,Gangplank,Ahri","Kalista,Lucian,Sejuani,Sylas,Lissandra","Ornn,Vi,Akali,Zeri,Lulu","Aatrox,Wukong,Taliyah,Sivir,Nautilus","Nate,MrLemon,WARANGELUS,Whitelotus,Newbie","Destroy (Yoon Jeong-min),MATAZ,Rainbow (Kim So...",0.0,4.0,0.0,1.0,2.0,9.0,47461.0,58407.0,6.0,15.0,2.0,0.0,0.0,1.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_2_1,LLA/2023 Season/Opening Promotion_Qualifiers_2,None,0,LLA
4,LLA/2023 Season/Opening Promotion,The Kings,Globant Emerald,The Kings,Globant Emerald,2022-09-11 21:01:00,2,0,1,32:47,32.783333,"Kalista,Lucian,Senna,Akali,Ornn","Trundle,Azir,Sejuani,Taliyah,Gangplank","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Aatrox,Wukong,LeBlanc,Sivir,Nautilus","Destroy (Yoon Jeong-min),MATAZ,Rainbow (Kim So...","Nate,MrLemon,WARANGELUS,Whitelotus,Newbie",3.0,1.0,1.0,0.0,9.0,3.0,61630.0,53630.0,19.0,10.0,1.0,1.0,2.0,0.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_2_2,LLA/2023 Season/Opening Promotion_Qualifiers_2,None,0,LLA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,CBLOL/2023 Season/Split 1,Los Grandes,RED Canids,RED Canids,Los Grandes,2023-01-22 16:25:00,0,1,2,37:26,37.433333,"Lulu,Lucian,Ryze,Ezreal,Jhin","Ashe,Heimerdinger,Yuumi,Nautilus,Viktor","Jayce,Sejuani,Kassadin,Varus,Lux","Camille,Maokai,Yone,Sivir,Karma","HiRit,Ranger,Lava,Netuno,zay","Boal,Aegis (Gabriel Lemos),Avenger (Adriano Pe...",2.0,4.0,0.0,2.0,3.0,8.0,60671.0,71546.0,12.0,23.0,0.0,2.0,0.0,1.0,13.1,CBLOL/2023 Season/Split 1_Week 1_6_1,CBLOL/2023 Season/Split 1_Week 1_6,None,0,CBLOL
6,CBLOL/2023 Season/Split 1,FURIA,INTZ,INTZ,FURIA,2023-01-22 17:26:00,0,1,2,33:12,33.200000,"Jayce,Zeri,Yuumi,Fiora,Jax","Ryze,Sejuani,Vi,Olaf,Gwen","Gnar,Wukong,Sylas,Lucian,Nami","K'Sante,Maokai,Akali,Caitlyn,Ashe","fNb,Goot